In [1]:
import json
import csv

In [2]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def loss_calculation_rouge(petitioner_argument,petitioner_version):
    petitioner_scores=scorer.score(petitioner_argument,petitioner_version)
    score_petitioner=petitioner_scores['rougeL']
    return score_petitioner.fmeasure

In [21]:
from nltk.translate.meteor_score import meteor_score
from bert_score import score
from nltk.tokenize import word_tokenize

def  loss_calculation_meteor(petitioner_argument,petitioner_version):
    #petitioner_argument=petitioner_argument.replace('"'," ")
    reference_sentences = petitioner_argument.split('. ')
    candidate_sentences = petitioner_version.split('. ')
    # Compute METEOR score for each sentence pair
    scores = []
    for ref, cand in zip(reference_sentences, candidate_sentences):
        ref= word_tokenize(ref)
        cand = word_tokenize(cand)
        scores.append(meteor_score([ref], cand))  # Pass reference as a list
    # Average METEOR score across all sentence pairs   
    average_meteor = sum(scores) / len(scores)
    #print(average_meteor)
    return average_meteor

/home/subinay/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# import json
# from transformers import BertTokenizer, BertForMaskedLM, BertModel
# from bert_score import BERTScorer
# import torch
# from bert_score import score

# def loss_calculation_bert(petitioner_argument,petitioner_version):
#     petitioner_argument=[petitioner_argument]
#     petitioner_version=[petitioner_version]
#     # F1= scorer.score(petitioner_argument,petitioner_version)
#     # F1= torch.cat(F1)
#     # F1_prosecutor=F1.mean().tolist()
#     P, R, F1 = score(petitioner_argument, petitioner_version, lang="en", verbose=True)
#     #print(f"F1 Score: {F1.item():.4f}")
#     return F1.item()

In [8]:
from datasets import load_metric
bertscore = load_metric("bertscore",trust_remote_code=True)
def loss_calculation_bert(actual, pred):
    #actual and pred are the strings.
    predictions = [pred]
    references = [actual]
    metrics = {}
    bert_score = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-uncased",lang="en")
    score=bert_score["f1"][0]
    return score

/home/subinay/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_35178/1899176948.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore = load_metric("bertscore",trust_remote_code=True)


In [9]:
import math 
def argument_score(gt_arguments,gen_arguments):
    #print(len(gt_argument),len(gen_argument))
    score=[]
    for i in range(len(gen_arguments)):
        arguments_score=[]
        for j in range(len(gt_arguments)):
            argument_score= loss_calculation_bert(gt_arguments[j],gen_arguments[i])
            #argument_score= loss_calculation_meteor(gt_arguments[j],gen_arguments[i])*math.exp(-abs(i-j))
            arguments_score.append(argument_score)
        score.append(max(arguments_score,default=0))
    print(score)
    return sum(score)/len(score)
    

In [10]:
def main_petitioner():
    json_test=open("/home/subinay/Documents/data/Argument/Langgraph/crime_test_docs_fact.json")
    jtest=json.load(json_test)
    test_docs=list(jtest.keys())
    json_petitioner_ground_truth=open("/home/subinay/Documents/data/Argument/Langgraph/GPT_4_o/Judge_Petitioner_Argument_test_doc_evalutation.json")
    petitioner_argument_gt=json.load(json_petitioner_ground_truth)
    json_petitioner_generated=open("/home/subinay/Documents/data/Argument/Langgraph/GPT_4_o/dynamic_fewshot .json")
    petitioner_argument_gen=json.load(json_petitioner_generated)
    print(len(petitioner_argument_gen))
    judge_petitioner_score=[]
    for test_doc in test_docs[0:20]:
        test_doc_id=test_doc.split(".")[0]
        #print(test_doc_id)
        gt_argument=petitioner_argument_gt[test_doc]['petitioner']
        for i in range(len(petitioner_argument_gen)):
            if(test_doc==list(petitioner_argument_gen[i].keys())[0]):
                gen_argument=petitioner_argument_gen[i]
                gen_argument=gen_argument[test_doc]
                gen_argument=gen_argument[0]['plaintiff']
                score=argument_score(gt_argument,gen_argument)
                print(f"score of {test_doc} is {score}")
                judge_petitioner_score.append(score)
    print(sum(judge_petitioner_score)/len(judge_petitioner_score))
    return None
                

In [11]:
main_petitioner()

100


/home/subinay/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/subinay/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2025-02-16 13:52:34.123460: E external/local_xla/

[0.5645712018013, 0.46281489729881287, 0.5014809370040894, 0.5379979610443115, 0.5501946806907654]
score of 2011_11-5683.txt is 0.5234119355678558
[0.5522663593292236, 0.5024847388267517, 0.4965262711048126, 0.5101139545440674, 0.5092814564704895]
score of 2008_06-11206.txt is 0.514134556055069
[0.488341748714447, 0.4858871400356293, 0.4712347090244293, 0.5687168836593628, 0.5084539651870728]
score of 2001_00-1214.txt is 0.5045268893241882
[0.5658713579177856, 0.5235474705696106, 0.5309520363807678, 0.4951764941215515, 0.5279427170753479]
score of 2007_06-6330.txt is 0.5286980152130127
[0.49200335144996643, 0.43921956419944763, 0.45459187030792236, 0.47978588938713074, 0.47809237241744995]
score of 2012_11-9540.txt is 0.46873860955238345
[0.5595468878746033, 0.5066109895706177, 0.5389676094055176, 0.6086161136627197, 0.5421375632286072]
score of 2011_11-210.txt is 0.5511758327484131
[0.5308334231376648, 0.5510984659194946, 0.5698747038841248, 0.5357958674430847, 0.535017728805542]
scor

In [35]:
import json
def main_respondent():
    json_test=open("/home/subinay/Documents/data/Argument/Langgraph/crime_test_docs_fact.json")
    jtest=json.load(json_test)
    test_docs=list(jtest.keys())
    json_respondent_ground_truth=open("/home/subinay/Documents/data/Argument/Langgraph/GPT_4_o/Judge_Respondent_Argument_test_doc_evalutation.json")
    respondent_argument_gt=json.load(json_respondent_ground_truth)
    json_respondent_generated=open("/home/subinay/Documents/data/Argument/Langgraph/GPT_4_o/round_5_courtroom_simulation_fewshot_local_Maxima_OpenAI.json")
    respondent_argument_gen=json.load(json_respondent_generated)
    print(len(respondent_argument_gen))
    judge_respondent_score=[]
    for test_doc in test_docs[0:50]:
        test_doc_id=test_doc.split(".")[0]
        #print(test_doc_id)
        gt_argument=respondent_argument_gt[test_doc]['respondent']
        for i in range(len(respondent_argument_gen)):
            if(test_doc==list(respondent_argument_gen[i].keys())[0]):
                gen_argument=respondent_argument_gen[i]
                gen_argument=gen_argument[test_doc]
                gen_argument=gen_argument[1]['respondant']
                score=argument_score(gt_argument,gen_argument)
                print(f"score of {test_doc} is {score}")
                judge_respondent_score.append(score)
    print(sum(judge_respondent_score)/len(judge_respondent_score))
    return None

In [36]:
main_respondent()

100
[0.5257692337036133, 0.49726253747940063, 0.5111915469169617, 0.515579342842102, 0.5132235884666443]
score of 2011_11-5683.txt is 0.5126052498817444
[0.5453711748123169, 0.4925045073032379, 0.5142658352851868, 0.5207984447479248, 0.5296632051467896]
score of 2008_06-11206.txt is 0.5205206334590912
[0.45690202713012695, 0.47037532925605774, 0.490896075963974, 0.49605411291122437, 0.5121098756790161]
score of 2001_00-1214.txt is 0.48526748418807986
[0.5643320083618164, 0.5049118399620056, 0.5443766713142395, 0.5019979476928711, 0.5378677845001221]
score of 2007_06-6330.txt is 0.530697250366211
[0.5014247298240662, 0.5176771879196167, 0.5104544162750244, 0.5016924142837524, 0.5092584490776062]
score of 2012_11-9540.txt is 0.5081014394760132
[0.5346806645393372, 0.5099309682846069, 0.5159433484077454, 0.5418190956115723, 0.5510676503181458]
score of 2011_11-210.txt is 0.5306883454322815
[0.5404117703437805, 0.5484155416488647, 0.4854696989059448, 0.5431364178657532, 0.5143646597862244]